In [1]:
from datetime import datetime, timedelta
import glob
import numpy as np
import pandas as pd
from sustain_drag_2020.fetch import fetch_nov2020
from sustain_drag_2020.irgason import read_irgason_from_toa5
from sustain_drag_2020.udm import clean_elevation_from_udm, read_udm_from_toa5
#from sustain_drag_2020.wavewire import read_wavewire_from_toa5

def read_wavewire_from_toa5(filenames):
    """Reads data from wave wire output file(s) in TOA5 format.
    If filenames is a string, process a single file. If it is
    a list of strings, process files in order and concatenate."""
    if type(filenames) is str:
        print('Reading ', filenames)
        data = [line.rstrip() for line in open(filenames).readlines()[4:]]
    elif type(filenames) is list:
        data = []
        for filename in filenames:
            print('Reading ', os.path.basename(filename))
            data += [line.rstrip() for line in open(filename).readlines()[4:]]
    else:
        raise RuntimeError('filenames must be string or list')

    times = []
    d = {'w1': [], 'w2': [], 'w3': [], 'w4': [], 'd1': [], 'd2': [], 'd3': [], 'd4': []}

    print('Processing wave wire time series..')

    for line in data:
        line = line.replace('"', '').split(',')
        timestr = line[0]
        if len(timestr) == 19:
            time = datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')
        elif len(timestr) == 21:
            time = datetime.strptime(timestr[:19], '%Y-%m-%d %H:%M:%S')
            time += timedelta(seconds=float(timestr[-2:]))
        else:
            time = datetime.strptime(timestr[:19], '%Y-%m-%d %H:%M:%S')
            time += timedelta(seconds=float(timestr[-3:]))
        times.append(time)
        d['w1'].append(float(line[2].strip('"')))
        d['w2'].append(float(line[3].strip('"')))
        d['w3'].append(float(line[4].strip('"')))
        d['w4'].append(float(line[5].strip('"')))
        d['d1'].append(float(line[6].strip('"')))
        d['d2'].append(float(line[7].strip('"')))
        d['d3'].append(float(line[8].strip('"')))
        d['d4'].append(float(line[9].strip('"')))
    for key in d.keys():
        d[key] = np.array(d[key])
        for i in range(1, d[key].size -1, 1):
            if d[key][i] < 0.2:
                d[key][i] = 0.5 * (d[key][i-1] + d[key][i+1])

    return np.array(times), d

import xarray as xr

#input data here
DATAPATH = '/Users/peisen/Downloads/20201118'
RUN_SECONDS = 600
FREQUENCY = 20
FREQUENCY_PRESSURE = 10
NUM_RUNS = 11

start_time = datetime(2020, 11, 18, 15, 21)
end_time = start_time + NUM_RUNS * timedelta(seconds=RUN_SECONDS)

# IRGASON
files = glob.glob(DATAPATH + '/TOA5_SUSTAIN_Wind.FAST*.dat')
print((files))
files.sort()
time, irg1, irg2 = read_irgason_from_toa5(files, valid_flag=11)

mask = (time >= start_time) & (time <= end_time)

ttime = time[mask]
print(time)
irg_seconds = np.array([(t - start_time).total_seconds() for t in ttime])
print(irg_seconds.max)
print(irg_seconds)

['/Users/peisen/Downloads/20201118/TOA5_SUSTAIN_Wind.FAST_20_2020_11_18_1700.dat', '/Users/peisen/Downloads/20201118/TOA5_SUSTAIN_Wind.FAST_21_2020_11_18_1800.dat', '/Users/peisen/Downloads/20201118/TOA5_SUSTAIN_Wind.FAST_18_2020_11_17_1725.dat', '/Users/peisen/Downloads/20201118/TOA5_SUSTAIN_Wind.FAST_19_2020_11_18_1600.dat', '/Users/peisen/Downloads/20201118/TOA5_SUSTAIN_Wind.FAST_22_2020_11_18_1900.dat']
Reading  TOA5_SUSTAIN_Wind.FAST_18_2020_11_17_1725.dat
Reading  TOA5_SUSTAIN_Wind.FAST_19_2020_11_18_1600.dat
Reading  TOA5_SUSTAIN_Wind.FAST_20_2020_11_18_1700.dat
Reading  TOA5_SUSTAIN_Wind.FAST_21_2020_11_18_1800.dat
Reading  TOA5_SUSTAIN_Wind.FAST_22_2020_11_18_1900.dat
Processing IRGASON time series..
[datetime.datetime(2020, 11, 17, 17, 25, 47, 150000)
 datetime.datetime(2020, 11, 17, 17, 25, 47, 200000)
 datetime.datetime(2020, 11, 17, 17, 25, 47, 250000) ...
 datetime.datetime(2020, 11, 18, 19, 59, 59, 900000)
 datetime.datetime(2020, 11, 18, 19, 59, 59, 950000)
 datetime.da

In [2]:
seconds = np.linspace(0, irg_seconds.max(), int(irg_seconds.max() * FREQUENCY) + 1, endpoint=True)
import os
fan = (seconds // RUN_SECONDS) * 5
fan[-1] = 50

u = np.interp(seconds, irg_seconds, irg2['u'][mask])
v = np.interp(seconds, irg_seconds, irg2['v'][mask])
w = np.interp(seconds, irg_seconds, irg2['w'][mask])
T = np.interp(seconds, irg_seconds, irg2['T'][mask])

# Wave wires
files = glob.glob(DATAPATH + '/TOA5_OSSwave*.dat')
files.sort()

print(type(files))

<class 'list'>


In [3]:
import pandas as pd
time, data = read_wavewire_from_toa5(files)

mask = (time >= start_time) & (time <= end_time)

ttime = time[mask]
ww_seconds = np.array([(t - start_time).total_seconds() for t in ttime])

eta_w = np.zeros((4, seconds.size))
eta_w[0,:] = np.interp(seconds, ww_seconds, data['w2'][mask])
eta_w[1,:] = np.interp(seconds, ww_seconds, data['w4'][mask])
eta_w[2,:] = np.interp(seconds, ww_seconds, data['w1'][mask])
eta_w[3,:] = np.interp(seconds, ww_seconds, data['w3'][mask])

for n in range(4):
    eta_w[n,:] -= np.mean(eta_w[n, seconds < RUN_SECONDS])

fetch_w = fetch_nov2020['wave_wire']
import pandas as pd

Reading  TOA5_OSSwaveX4.elev_272_2020_11_17_1725.dat
Reading  TOA5_OSSwaveX4.elev_273_2020_11_18_1600.dat
Reading  TOA5_OSSwaveX4.elev_274_2020_11_18_1700.dat
Reading  TOA5_OSSwaveX4.elev_275_2020_11_18_1800.dat
Reading  TOA5_OSSwaveX4.elev_276_2020_11_18_1900.dat
Processing wave wire time series..


In [4]:
# UDM
import pandas as pd
files = glob.glob(DATAPATH + '/TOA5_SUSTAIN_ELEV*.dat')
files.sort()
time, u1, u2, u3, u4, u5, u6 = read_udm_from_toa5(files)

mask = (time >= start_time) & (time <= end_time)
ttime = time[mask]
udm_seconds = np.array([(t - start_time).total_seconds() for t in ttime])

eta_u = np.zeros((5, seconds.size))
eta_u[0,:] = np.interp(seconds, udm_seconds, u3[mask])
eta_u[1,:] = np.interp(seconds, udm_seconds, u4[mask])
eta_u[2,:] = np.interp(seconds, udm_seconds, u1[mask])
eta_u[3,:] = np.interp(seconds, udm_seconds, u6[mask])
eta_u[4,:] = np.interp(seconds, udm_seconds, u5[mask])

def clean_elevation_from_udm(x, max_value=0.3, max_jump=0.08, interpolation_method='polynomial'):
    import pandas as pd
    xx = np.mean(x[:20 * 600]) - x[:] # offset first 10 minutes
    xdiff = xx[1:] - xx[:-1]
    xx[1:][np.abs(xdiff) > max_jump] = np.nan # difference between 2 points should not exceed max_jump
    xx[np.abs(xx) > max_value] = np.nan # elevation should not go below min_trough
    return np.array(pd.DataFrame(data=xx).interpolate(method=interpolation_method, order=3))[:,0]
for n in range(5):
    eta_u[n,:] = clean_elevation_from_udm(eta_u[n,:])

fetch_u = fetch_nov2020['udm']


Reading  TOA5_SUSTAIN_ELEVx6d_20Hz.ELEV_18_2020_11_17_1725.dat
Reading  TOA5_SUSTAIN_ELEVx6d_20Hz.ELEV_19_2020_11_18_1600.dat
Reading  TOA5_SUSTAIN_ELEVx6d_20Hz.ELEV_20_2020_11_18_1700.dat
Reading  TOA5_SUSTAIN_ELEVx6d_20Hz.ELEV_21_2020_11_18_1800.dat
Reading  TOA5_SUSTAIN_ELEVx6d_20Hz.ELEV_22_2020_11_18_1900.dat


/Users/peisen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in greater
/Users/peisen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in greater


In [5]:
# Static pressure
data = pd.read_csv(DATAPATH + '/scanivalve_mps_' + start_time.strftime('%Y%m%d') + '_windonly.csv')
time = data['FTime']

fan_p = (time // RUN_SECONDS) * 5
fan[-1] = 50

fetch_p = fetch_nov2020['static_pressure']

p = np.zeros((9, time.size))

p[0,:] = data['33Press'][:]
p[1,:] = data['34Press'][:]
p[2,:] = data['35Press'][:]
p[3,:] = data['36Press'][:]
p[4,:] = data['37Press'][:]
p[5,:] = data['38Press'][:]
p[6,:] = data['39Press'][:]
p[7,:] = data['40Press'][:]
p[8,:] = data['41Press'][:]

# Create dataset
ds = xr.Dataset(
    {
        'fan': ('time', fan), 
        'fan_p': ('time_p', fan_p), 
        'u': ('time', u), 
        'v': ('time', v), 
        'w': ('time', w), 
        'T': ('time', T),
        'eta_w': (['fetch_w', 'time'], eta_w),
        'eta_u': (['fetch_u', 'time'], eta_u),
        'p': (['fetch_p', 'time_p'], p),
    },
    coords = {
        'time': seconds,
        'time_p': time.to_numpy(),
        'fetch_w': [1.5300,9.3000,10.8000,14],
        'fetch_u': fetch_u,
        'fetch_p': fetch_p
    }
)

# Add metadata
ds['time'].attrs['name'] = 'Time since start of experiment'
ds['time'].attrs['units'] = 's'
ds['time_p'].attrs['name'] = 'Time since start of experiment, static pressure only'
ds['time_p'].attrs['units'] = 's'
ds['fetch_w'].attrs['name'] = 'Fetch of wave wires'
ds['fetch_w'].attrs['units'] = 'm'
ds['fetch_u'].attrs['name'] = 'Fetch of UDM'
ds['fetch_u'].attrs['units'] = 'm'
ds['fan'].attrs['name'] = 'Fan speed'
ds['fan'].attrs['units'] = 'Hz'
ds['fan_p'].attrs['name'] = 'Fan speed in pressure time coordinate'
ds['fan_p'].attrs['units'] = 'Hz'
ds['u'].attrs['name'] = 'Along-tank velocity from IRGASON'
ds['u'].attrs['units'] = 'm/s'
ds['u'].attrs['fetch'] = 9.55
ds['v'].attrs['name'] = 'Cross-tank velocity from IRGASON'
ds['v'].attrs['units'] = 'm/s'
ds['v'].attrs['fetch'] = 9.55
ds['w'].attrs['name'] = 'Vertical velocity from IRGASON'
ds['w'].attrs['units'] = 'm/s'
ds['w'].attrs['fetch'] = 9.55
ds['T'].attrs['name'] = 'Air temperature from IRGASON'
ds['T'].attrs['units'] = 'K'
ds['T'].attrs['fetch'] = 9.55
ds['p'].attrs['name'] = 'Static pressure at the ceiling'
ds['p'].attrs['units'] = 'Pa'
ds['eta_w'].attrs['name'] = 'Water elevation from wave wires'
ds['eta_w'].attrs['units'] = 'm'
ds['eta_u'].attrs['name'] = 'Water elevation from UDM'
ds['eta_u'].attrs['units'] = 'm'

ds.attrs['experiment_name'] = 'wind-only_fresh-water_20201118'
ds.attrs['experiment_time'] = start_time.strftime('%Y-%m-%d_%H:%M:%S')
ds.attrs['water_type'] = 'fresh'
ds.attrs['initial_water_depth'] = 0.8
ds.attrs['institution'] = 'University of Miami'
ds.attrs['facility'] = 'SUSTAIN Laboratory'
ds.attrs['tank'] = 'SUSTAIN'
ds.attrs['contact'] = 'Milan Curcic <mcurcic@miami.edu>'
import os
os.chdir('/Users/peisen/Downloads/20201118/NC')
ds.to_netcdf('sustain_drag_' + start_time.strftime('%Y%m%d')  + '.nc', 'w', 'NETCDF4')
print(start_time)
print(end_time)

2020-11-18 15:21:00
2020-11-18 17:11:00
